## 7.3 Queues

A stack is a sequence where items are added and removed from the same end.
A queue is a sequence where items are added to one end and removed from the other.

### 7.3.1 Queues

People queue when boarding planes; cars queue at drive-ins.
People and cars join the queue at the end and leave it at the front.
Like stacks, queues are sequences of items ordered by arrival order, but
contrary to stacks, the item arriving first is also the first to be processed:
the 'oldest' item is at the bottom of the stack but at the front of the queue.
A **queue** is a **first in, first out (FIFO)** sequence.
The operations are very similar to a stack's push, pop, and peek.

Operation | Effect | Algorithm in English
:-|:-|:-
new  | create an empty queue *q* | let *q* be an empty queue
length | the number of items in *q*  | │*q*│
enqueue | add an item to the back of *q*  | add *item* to *q*
dequeue | remove the front item, if *q* isn't empty |  dequeue *q*
front | access the front item of, if *q* isn't empty | front of *q*

Alternatively, the dequeue operation could remove and return the front item,
but this wouldn't make the front operation redundant. Why?

___

With stacks, we can inspect the top item without 'disturbing' the stack,
by popping it and pushing it immediately back.
We can't inspect the front item of a queue with a dequeue followed by an
enqueue operation: the front item would end up at the back of the queue.

#### Exercise 7.3.1

How would you represent a to-do list of tasks:
with a general sequence, a stack, or a queue?

[Hint](../31_Hints/Hints_07_3_01.ipynb)
[Answer](../32_Answers/Answers_07_3_01.ipynb)

#### Exercise 7.3.2 (optional)

Define the queue operations in the same way as the
[stack operations](../07_Ordered/07_1_stack.ipynb#7.1-Stacks), using sequence notation.
The operations are similar, so you may wish to define only one or two of them.

### 7.3.2 Queues with Python lists

Python's versatile lists can not only simulate stacks but also queues.

In [1]:
queue = []
queue.append('Alice')   # Alice arrives first
queue.append('Bob')
print('Next person served:', queue.pop(0))
queue.append('Clara')
print('Next person served:', queue.pop(0))
print('Next person served:', queue.pop(0))
print('People still waiting?', len(queue) > 0)

Next person served: Alice
Next person served: Bob
Next person served: Clara
People still waiting? False


This approach puts the front of the queue at index&nbsp;0 and therefore uses
`append` as the enqueue operation and  `pop(0)` as the dequeue operation.
The downside is that the latter takes time linear in the length of the list,
because each remaining item is shifted one position down.

We could instead have the front of the queue at the last index.
That would make dequeuing with `pop(-1)` take constant time,
but enqueuing with `insert(0, item)` would take linear time.

To sum up, using Python lists and their methods makes one queue operation
take linear time, which is fine if the queue never gets too long.

### 7.3.3 Queues with linked lists

The queue ADT can also be implemented with a linked list.
If the front item is in the head node, then the dequeue operation removes the
head node and the enqueue operation adds a node at the back of the linked list.
We can traverse the linked list in linear time to find the last node and make it
point to the new enqueued node, but by keeping a pointer to the last node we can
enqueue a new node in constant time.

Likewise, we can compute the length of a queue in linear time,
by traversing the linked list and counting nodes, but by keeping an integer
with the current size we can return the length in constant time.

In both cases, we use a space-time tradeoff: storing additional data, and keeping
it up to date as the queue operations are executed, leads to a better run-time.
Here's the code.

In [2]:
# this code is also in m269_queue.py

class Queue:
    """A last-in, first-out sequence of objects, implemented with a linked list."""

    class Node:
        """A node in a linked list."""

        def __init__(self, item: object):
            """Initialise the node with the given item."""
            self.item = item
            self.next = None

    def __init__(self):
        """Initialise the queue to be empty."""
        self.head = None
        self.last = None
        self.length = 0

    def size(self) -> int:
        """Return the number of items in the queue."""
        return self.length

    def front(self) -> object:
        """Return the item at the front of the queue.

        Preconditions: self.size() > 0
        """
        return self.head.item

    def enqueue(self, item) -> None:
        """Add item to the back of the queue."""
        node = Queue.Node(item)
        if self.length == 0:
            self.head = node
            self.last = node
        else:
            self.last.next = node
            self.last = node
        self.length = self.length + 1

    def dequeue(self):
        """Remove and return the item at the front of the queue.

        Preconditions: self.size() > 0
        """
        item = self.front()
        self.head = self.head.next
        if self.head == None:
            self.last = None
        self.length = self.length - 1
        return item

Here's the same example, using the class:

In [3]:
queue = Queue()
queue.enqueue('Alice')   # Alice arrives first
queue.enqueue('Bob')
print('Next person served:', queue.dequeue())
queue.enqueue('Clara')
print('Next person served:', queue.dequeue())
print('Next person served:', queue.dequeue())
print('People still waiting?', queue.size() > 0)

Next person served: Alice
Next person served: Bob
Next person served: Clara
People still waiting? False


<div class="alert alert-info">
<strong>Info:</strong> In Java, queue operations are defined by interface <code>Deque</code>,
which implements double-ended queues, an extension of queues.
The interface is usually implemented by class <code>ArrayDeque</code> or <code>LinkedList</code>.
Both the interface and the classes are in package <code>java.util</code>.
</div>

#### Exercise 7.3.3

If the queue were stored in the opposite order, with the last item in the head and
the first item in the last node of the linked list, could we still implement
the queue's operations in constant time?

_Write your answer here._

[Hint](../31_Hints/Hints_07_3_03.ipynb)
[Answer](../32_Answers/Answers_07_3_03.ipynb)

### 7.3.4 Using queues

Consider *n* children in a circle, numbered clockwise from 1 to *n*.
Alice is in the centre. She points at the first child and starts reciting:

> Eeny, meeny, miny, moe,\
> Catch a tiger by the toe.\
> If he hollers, let him go,\
> Eeny, meeny, miny, moe.

For each syllable, Alice points at a child, going clockwise.
For example, with *n* = 3, she would point successively at
children 1 (ee) 2 (ny) 3 (mee) 1 (ny) 2 (mi) 3 (ny) 1 (moe) for the first line.
The child pointed to on the last syllable, the second 'moe', leaves the circle.
The reciting and counting starts again on the next child.
After going *n* − 1 times through the rhyme, one child is left in the circle.
We want to know which child is that.

<div class="alert alert-info">
<strong>Info:</strong> This is a version of the
<a href="https://en.wikipedia.org/wiki/Josephus_problem">Josephus problem</a>.
</div>

#### Exercise 7.3.4

For this problem, we can represent a circle of children as a queue. How?

_Write your answer here._

[Hint](../31_Hints/Hints_07_3_04.ipynb)
[Answer](../32_Answers/Answers_07_3_04.ipynb)

#### Exercise 7.3.5

Given the number of children *n*, we want to know the number of
the last remaining child. Outline an algorithm to compute that number.

_Write your answer here._

[Hint](../31_Hints/Hints_07_3_05.ipynb)
[Answer](../32_Answers/Answers_07_3_05.ipynb)

#### Exercise 7.3.6

Implement the algorithm you outlined by completing the following function, using the `Queue` class.

In [4]:
%run -i ../m269_queue

from algoesup import test

def counting_rhyme(n: int) -> int:
    """Return which child from 1 to n remains last in the circle.

    Preconditions: n > 0
    """
    pass

counting_rhyme_tests = [
    # case,         n,  last child
    ['1 child',     1,          1],
    ['2 children',  2,          1],
    ['3 children',  3,          2]
]

test(counting_rhyme, counting_rhyme_tests)

[Hint](../31_Hints/Hints_07_3_06.ipynb)
[Answer](../32_Answers/Answers_07_3_06.ipynb)

#### Exercise 7.3.7

What is the complexity of the algorithm as implemented in the solution to the previous exercise?

_Write your answer here._

[Hint](../31_Hints/Hints_07_3_07.ipynb)
[Answer](../32_Answers/Answers_07_3_07.ipynb)

⟵ [Previous section](07_2_stack_usage.ipynb) | [Up](07-introduction.ipynb) | [Next section](07_4_priority_queue.ipynb) ⟶